# Fortune 1000 companies

## 1) Read file

soure of data file: https://data.world/eruditepanda/fortune-1000-list-2018

In [1]:
import pandas as pd
import numpy as np
f1000 = pd.read_csv('fortune1000-final.csv', encoding = 'Latin-1')
print('Dataframe has {row} rows and {col} columns.'.format(row = f1000.shape[0], col = f1000.shape[1]))

Dataframe has 1000 rows and 19 columns.


show some basic dataset information

In [2]:
f1000.info()
#f1000.columns
#f1000.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
Rank                              1000 non-null int64
Title                             1000 non-null object
Previous Rank                     937 non-null float64
 Revenues ($M)                    1000 non-null object
Revenue Change                    995 non-null object
 Profits ($M)                     998 non-null object
Profit Change                     844 non-null object
 Assets ($M)                      1000 non-null object
 Mkt Value as of 3/29/18 ($M)     951 non-null object
Employees                         1000 non-null int64
CEO                               1000 non-null object
CEO Title                         1000 non-null object
Sector                            1000 non-null object
Industry                          1000 non-null object
Years on Fortune 500 List         500 non-null float64
City                              999 non-null object
State                

In [3]:
f1000.head(3)

,Rank,Title,Previous Rank,Revenues ($M),Revenue Change,Profits ($M),Profit Change,Assets ($M),Mkt Value as of 3/29/18 ($M),Employees,CEO,CEO Title,Sector,Industry,Years on Fortune 500 List,City,State,Latitude,Longitude
0,1,Walmart,1.0,"$500,343.00",3%,"$9,862.00",-28%,"$204,522.00","$263,563.00",2300000,C. Douglas McMillon,"President, Chief Executive Officer & Director",Retailing,General Merchandisers,24.0,Bentonville,AR,36.37285,(94.20882)
1,2,Exxon Mobil,4.0,"$244,363.00",17%,"$19,710.00",151%,"$348,691.00","$316,157.00",71200,Darren W. Woods,Chairman & Chief Executive Officer,Energy,Petroleum Refining,24.0,Irving,TX,32.81402,(96.94889)
2,3,Berkshire Hathaway,2.0,"$242,137.00",8%,"$44,940.00",87%,"$702,095.00","$492,008.00",377000,Warren E. Buffett,"Chairman, President & Chief Executive Officer",Financials,Insurance: Property and Casualty (Stock),24.0,Omaha,NE,41.25654,(95.93450)


Observations:
- column names (header) contain spaces
- 'money' columns (revenues, profits, ...) to be changed to a numeric type
- 'change' columns (revenue change, profit change, ...) also to be changed to a numeric type
- `previous rank` might be converted to a integer
- check `sector` and `industry`, values should be lists
- check `city` and `state`
- check spatial coordinates, values should be numeric

## 2) Data cleansing

### 2.1) header

We modify the header such that
- the column names are all lower case,
- leading and trailing spaces are removed, and
- spaces between words are replaced by underscores

Note: additional modifications are done below

In [4]:
f1000.rename(columns = lambda x: x.lower().strip().replace(' ', '_'), inplace = True)

### 2.2) columns: rank, title, and previous rank

In [5]:
# rename the 'previous rank' column
f1000.rename(columns = {'previous_rank':'prev_rank'}, inplace = True)

In [6]:
f1000['rank'].describe()

count    1000.000000
mean      500.499000
std       288.818771
min         1.000000
25%       250.750000
50%       500.500000
75%       750.250000
max      1000.000000
Name: rank, dtype: float64

In [7]:
f1000['rank'].value_counts().head()

692     2
1000    1
327     1
340     1
339     1
Name: rank, dtype: int64

In [8]:
f1000[f1000['rank']==692]

,rank,title,prev_rank,revenues_($m),revenue_change,profits_($m),profit_change,assets_($m),mkt_value_as_of_3/29/18_($m),employees,ceo,ceo_title,sector,industry,years_on_fortune_500_list,city,state,latitude,longitude
691,692,Brinks,730.0,"$3,347.00",11%,$16.70,-52%,"$3,060.00","$3,609.00",62150,Douglas A. Pertz,"President, Chief Executive Officer & Director",Business Services,Diversified Outsourcing Services,NaN,Richmond,VA,37.54072,(77.43605)
692,692,Meritor,730.0,"$3,347.00",11%,$16.70,-52%,"$3,060.00","$3,609.00",62150,Douglas A. Pertz,"President, Chief Executive Officer & Director",Business Services,Diversified Outsourcing Services,NaN,Richmond,VA,37.54072,(77.43605)


In [9]:
f1000['title'].describe()

count                               1000
unique                              1000
top       Expeditors Intl. of Washington
freq                                   1
Name: title, dtype: object

In [10]:
f1000['prev_rank'].describe()

count    937.000000
mean     485.988260
std      284.697048
min        1.000000
25%      239.000000
50%      481.000000
75%      731.000000
max      996.000000
Name: prev_rank, dtype: float64

In [11]:
len(f1000[f1000['prev_rank'].isnull()])
#f1000[f1000['prev_rank'].isnull()].shape[0]

63

In [12]:
f1000['prev_rank'].value_counts().head()

730.0    2
403.0    2
306.0    2
941.0    1
844.0    1
Name: prev_rank, dtype: int64

conclusion:
- `title` is unique and we can use it as an index
- `rank` has duplicates
- Brinks and Meritor show identical values (except title) and are both ranked equally (692)
- `previous rank` has 63 nulls (NaN) and duplicates
- `previous rank`'s datatype is float (not integer) as NaN is treated as a float
- pandas supports nullable integer data types experimentally (version 0.24.0)

### 2.3) index
We didn't set an index to the dataframe, so we can slice in the row-dimension only by row number (the order the file has been read). We'll set an index on the `title` column to enable slicing by title. Remember, there are duplicates in the `rank` column. Thus, we cannot use this column as index.

In [13]:
print('index before modification:\n{0}'.format(f1000.index))
f1000.set_index('title', verify_integrity = True, inplace = True)
#f1000.set_index('rank', inplace=True, verify_integrity=True) # runs in an error due to duplicates
#f1000.reset_index(inplace=True)
print('\nindex after modification:\n{0}'.format(f1000.index))

index before modification:
RangeIndex(start=0, stop=1000, step=1)

index after modification:
Index(['Walmart', 'Exxon Mobil', 'Berkshire Hathaway', 'Apple',
       'UnitedHealth Group', 'McKesson', 'CVS Health', 'Amazon.com', 'AT&T',
       'General Motors',
       ...
       'Superior Energy Services', 'VeriFone Systems', 'Childrens Place',
       'Tribune Media', 'Healthcare Services Group',
       'SiteOne Landscape Supply', 'Charles River Laboratories Intl',
       'CoreLogic', 'Ensign Group', 'HCP'],
      dtype='object', name='title', length=1000)


In [14]:
# example: slice by title
f1000.loc['Walmart':'Apple', ['title', 'sector']]

,title,sector
title,,
Walmart,NaN,Retailing
Exxon Mobil,NaN,Energy
Berkshire Hathaway,NaN,Financials
Apple,NaN,Technology


Note: `title` is now the index and no regular column anymore (see `drop` parameter in `df.set_index`)

### 2.4) monetary columns and changes

In [15]:
# rename the 'previous rank' column
f1000.rename(columns = { \
    'revenues_($m)'               : 'revenue_usd', \
    'revenue_change'              : 'revenue_chg', \
    'profits_($m)'                : 'profit_usd', \
    'profit_change'               : 'profit_chg', \
    'assets_($m)'                 : 'asset_usd', \
    'mkt_value_as_of_3/29/18_($m)': 'mkt_value_usd'
}, inplace = True)
f1000.columns

Index(['rank', 'prev_rank', 'revenue_usd', 'revenue_chg', 'profit_usd',
       'profit_chg', 'asset_usd', 'mkt_value_usd', 'employees', 'ceo',
       'ceo_title', 'sector', 'industry', 'years_on_fortune_500_list', 'city',
       'state', 'latitude', 'longitude'],
      dtype='object')

Statistics of columns before cleansing:

In [16]:
f1000[['revenue_chg', 'profit_chg', 'revenue_usd', 'profit_usd', 'asset_usd', 'mkt_value_usd']].describe()

,revenue_chg,profit_chg,revenue_usd,profit_usd,asset_usd,mkt_value_usd
count,995,844,1000,998,1000,951
unique,103,324,951,971,973,922
top,5%,28%,"$2,788.00",$30.00,"$1,733.00",$-
freq,68,12,3,2,3,5


In [17]:
#f1000['revenue_chg'].unique()
#f1000['profit_chg'].unique()

The percentage changes of revenue and profit are regular and clean. Thus, we only remove the percent sign, convert it to a float and divide it by 100.

In [18]:
pct_cols = ['revenue_chg', 'profit_chg']
for col in pct_cols:
    f1000[col] = f1000[col].str.replace('%', '').astype(float) / 100.0

In [19]:
#f1000['revenue_usd'].unique()
#f1000['profit_usd'].unique()
#f1000['asset_usd'].unique()
#f1000['mkt_value_usd'].unique()

However, the monetary colums are dirty but regular. We have to remove all blanks, the thousands separator, and the dollar sign. No value is given as `-` (minus), we convert it to 0. Negative values are surrounded by parenthesis. We convert it using a regular expression (the pattern will be re-used later). Finally, cast the string to a float and multiply it by 1 million.

In [20]:
money_cols = ['revenue_usd', 'profit_usd', 'asset_usd', 'mkt_value_usd']
re_pat_negnum = r'^\((?P<nbr>[0-9\.]+)\)$'
re_repl_negnum = lambda m: '-' + m.group('nbr')
for col in money_cols:
    f1000[col] = f1000[col].str.replace(' ', '').str.replace(',', '').str.replace('$', '').str.replace('-', '0')
    f1000[col] = f1000[col].str.replace(re_pat_negnum, re_repl_negnum).astype(float) * 1000000.0

Statistics of columns after cleansing:

In [21]:
f1000[['revenue_chg', 'profit_chg', 'revenue_usd', 'profit_usd', 'asset_usd', 'mkt_value_usd']].describe()

,revenue_chg,profit_chg,revenue_usd,profit_usd,asset_usd,mkt_value_usd
count,995.000000,844.000000,1.000000e+03,9.980000e+02,1.000000e+03,9.510000e+02
mean,0.114613,0.276623,1.437820e+10,1.111773e+09,4.602315e+10,2.625751e+10
std,0.332751,6.035319,3.083585e+10,3.477028e+09,1.992486e+11,6.602655e+10
min,-0.420000,-117.000000,1.848000e+09,-6.798000e+09,4.790000e+08,0.000000e+00
25%,0.020000,-0.212500,2.886500e+09,9.690000e+07,3.498250e+09,3.070000e+09
50%,0.060000,0.110000,5.417000e+09,3.210500e+08,8.370500e+09,8.523000e+09
75%,0.130000,0.570000,1.210875e+10,9.714500e+08,2.371225e+10,2.159350e+10
max,6.130000,105.570000,5.003430e+11,4.835100e+10,3.345529e+12,8.513180e+11


### 2.5) location: latitude, longitude

The latitude already is recognised as a float, longitude has to be converted.

In [22]:
f1000[['latitude', 'longitude']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, Walmart to HCP
Data columns (total 2 columns):
latitude     1000 non-null float64
longitude    1000 non-null object
dtypes: float64(1), object(1)
memory usage: 23.4+ KB


In [23]:
# re-use the regular expression pattern for negative numbers surrounded by paranthesis
f1000['longitude'] = f1000['longitude'].str.strip().str.replace(re_pat_negnum, re_repl_negnum).astype(float)

In [24]:
f1000[['latitude', 'longitude']].describe()

,latitude,longitude
count,1000.000000,1000.000000
mean,38.228050,-90.359699
std,4.643421,16.145941
min,18.422580,-157.858330
25%,35.207700,-96.698890
50%,39.693420,-86.583760
75%,41.499320,-77.177260
max,47.978980,-66.050950


### 2.6) location: city and state

In [25]:
f1000[['city', 'state']].describe()
#f1000[f1000['city'].isnull()]

,city,state
count,999,1000
unique,400,46
top,New York,CA
freq,70,118


In [26]:
f1000['city'].unique()[0:5]

array(['Bentonville', 'Irving', 'Omaha', 'Cupertino', 'Minnetonka'], dtype=object)

In [27]:
f1000['city'].value_counts().head()

New York    70
Houston     39
Chicago     27
Atlanta     21
Dallas      16
Name: city, dtype: int64

In [28]:
f1000['state'].unique()

array(['AR', 'TX', 'NE', 'CA', 'MN', 'RI', 'WA', 'MI', 'PA', 'OH', 'NY',
       'MA', 'IL', 'GA', 'NC', 'MO', 'IN', 'NJ', 'VA', 'Michigan', 'CT',
       'TN', 'ID', 'KY', 'MD', 'FL', 'OR', 'WI', 'CO', 'AZ', 'LA', 'IA',
       'OK', 'NV', 'KS', 'DE', 'AL', 'SC', 'ND', 'MS', 'UT', 'Puerto Rico',
       'NH', 'HI', 'VT', 'ME'], dtype=object)

We will convert `Michigan` to `MI` and show `Puerto Rico` as a country.

In [29]:
f1000['country'] = 'USA'
f1000.loc[f1000['state']=='Puerto Rico', 'country'] = 'Puerto Rico'
f1000.loc[f1000['state']=='Michigan',    'state']   = 'MI'
f1000.loc[f1000['state']=='Puerto Rico', 'state']   = np.nan

In [30]:
f1000['state'].unique()

array(['AR', 'TX', 'NE', 'CA', 'MN', 'RI', 'WA', 'MI', 'PA', 'OH', 'NY',
       'MA', 'IL', 'GA', 'NC', 'MO', 'IN', 'NJ', 'VA', 'CT', 'TN', 'ID',
       'KY', 'MD', 'FL', 'OR', 'WI', 'CO', 'AZ', 'LA', 'IA', 'OK', 'NV',
       'KS', 'DE', 'AL', 'SC', 'ND', 'MS', 'UT', nan, 'NH', 'HI', 'VT',
       'ME'], dtype=object)

In [31]:
f1000['country'].unique()

array(['USA', 'Puerto Rico'], dtype=object)

In [32]:
f1000.loc[f1000['country']=='Puerto Rico', 'state'].unique()

array([nan], dtype=object)

In [33]:
f1000.loc[f1000['country']=='USA', 'state'].unique()

array(['AR', 'TX', 'NE', 'CA', 'MN', 'RI', 'WA', 'MI', 'PA', 'OH', 'NY',
       'MA', 'IL', 'GA', 'NC', 'MO', 'IN', 'NJ', 'VA', 'CT', 'TN', 'ID',
       'KY', 'MD', 'FL', 'OR', 'WI', 'CO', 'AZ', 'LA', 'IA', 'OK', 'NV',
       'KS', 'DE', 'AL', 'SC', 'ND', 'MS', 'UT', 'NH', 'HI', 'VT', 'ME'], dtype=object)

### 2.7) secotor and industry

In [34]:
f1000['sector'].value_counts()

Financials                        155
Energy                            107
Technology                        103
Retailing                          77
Health Care                        71
Business Services                  53
Industrials                        49
Materials                          45
Wholesalers                        44
Transportation                     40
Food, Beverages &  Tobacco         37
Chemicals                          33
Household Products                 28
Engineering &  Construction        27
Hotels, Restaurants &  Leisure     26
Aerospace &  Defense               25
Media                              25
Motor Vehicles &  Parts            19
Apparel                            14
Food &  Drug Stores                12
Telecommunications                 10
Name: sector, dtype: int64

In [35]:
f1000['sector'] = f1000['sector'].str.replace(' &  ', ' & ')

In [36]:
f1000['industry'].value_counts()

Utilities: Gas and Electric                       35
Chemicals                                         33
Specialty Retailers: Other                        31
Insurance: Property and Casualty (Stock)          28
Industrial Machinery                              27
Commercial Banks                                  27
Semiconductors and Other Electronic Components    26
Aerospace and Defense                             25
Mining, Crude-Oil Production                      24
Wholesalers: Diversified                          24
Financial Data Services                           24
Securities                                        22
Real estate                                       22
Specialty Retailers: Apparel                      21
Entertainment                                     21
Motor Vehicles and Parts                          19
Insurance: Life, Health (stock)                   19
Food Consumer Products                            19
Energy                                        

No cleansing needed.

### 2.8) ceo and ceo title

In [37]:
f1000['ceo'].value_counts()

Gregory B. Maffei                2
Terry D. Bassham                 2
Jack Dorsey                      2
Douglas A. Pertz                 2
Curtis A. Morgan                 2
David M. Zaslav                  2
Vernon J. Nagel                  1
Kenneth A. Burdick               1
Arne M. Sorenson                 1
Carl S. Rubin                    1
Paul S. Galant                   1
Beth E. Mooney                   1
Andrï¿½ Calantzopoulos           1
John F. Barrett                  1
Thomas A. Kennedy                1
Michael H. Thaman                1
Michael K. Wirth                 1
Sanjay Mehrotra                  1
Jonathan H. Weis                 1
George J. Damiris                1
Andrew Wilson                    1
Ryan M. Lance                    1
Dan L. Batrack                   1
Gerry P. Smith                   1
Daniel J. Crowley                1
Donald E. Brandt                 1
Scott C. Donnelly                1
Mauricio Gutierrez               1
Ronald F. Clarke    

In [38]:
f1000['ceo_title'].value_counts()

President, Chief Executive Officer &  Director            400
Chairman &  Chief Executive Officer                       318
Chairman, President &  Chief Executive Officer            208
Chief Executive Officer &  Director                        28
Chief Executive Officer                                    15
Co-Chief Executive Officer &  Director                      3
Interim Chief Executive Officer &  Director                 3
Vice Chairman, President &  Chief Executive Officer         2
Chairman &  Co-Chief Executive Officer                      2
Vice Chairman &  Chief Executive Officer                    2
Co-Chairman &  Co-Chief Executive Officer                   2
Chairman &  Interim Chief Executive Officer                 2
President-Campbell Snacks                                   1
Co-Chairman &  Chief Executive Officer                      1
CEO-SS& C Technologies Holdings                             1
Co-Chairman, President &  Chief Executive Officer           1
CEO-Vist

In [39]:
f1000['ceo_title'] = f1000['ceo_title'].str.replace(' &  ', ' & ')

### 2.9) employees

In [40]:
f1000['employees'].describe()

count    1.000000e+03
mean     3.365251e+04
std      9.146179e+04
min      1.260000e+02
25%      6.400000e+03
50%      1.279950e+04
75%      2.922500e+04
max      2.300000e+06
Name: employees, dtype: float64